# Install required apt packages

In [ ]:
# @title Install required apt packages
!apt-get update
!apt-get install -y build-essential python3.11 python3.11-dev unzip pciutils net-tools libgl1
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1
!update-alternatives --install /usr/bin/python python /usr/bin/python3.11 1
!python3 --version
!python --version
!echo "-= Done. =-"

# Install ollama

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!echo "-= Done. =-"

# Install openwebui

In [ ]:
!pip install open-webui
!pip install ffmpeg
!echo "-= Done. =-"

# Start ollama

In [44]:
import subprocess

log_file = "ollama.log"
with open(log_file, "w") as f:
    subprocess.Popen(["ollama", "serve"], stdout=f, stderr=f)


# Show ollama logs

In [ ]:
!cat ollama.log

# Install cloudflare

In [ ]:
# @title Install cloudflare
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!rm -rf cloudflared-linux-amd64.deb
!echo "-= Done. =-"

# Start cloudflare tunnel and openwebui

In [ ]:
# @title Start cloudflare tunnel and openwebui
import subprocess
import threading
import time
import socket
import urllib.request

OPENWEBUI_PORT = 9999

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      cf_url = l[l.find("http"):]
      print("This is the URL to access ComfyUI:", cf_url, end='')

threading.Thread(target=iframe_thread, daemon=True, args=(OPENWEBUI_PORT,)).start()

!open-webui serve --port $OPENWEBUI_PORT

!echo "-= Done. =-"